In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.sparse import diags

from utils_1D import gaussian_process, sense, normalize, solve_Cahn_Hilliard, FNN, plot_2D

In [ ]:
class ONet(tf.keras.Model):
  
  def __init__(self, trunk, branch):
    super(ONet, self).__init__()
    
    self.trunk = trunk
    self.branch = branch

  def call(self, x_sensor, x, return_grad = False):

    #y_trunk = self.trunk(x)

    #x_sensor = np.expand_dims(x_sensor, 0)
    u_branch = self.branch(x_sensor)
    
    if return_grad:
        with tf.GradientTape(persistent=True) as g1:
            g1.watch(x)
            u_trunk = self.trunk(x)
            #y_out = tf.tensordot(y_branch, y_trunk, axes=([1], [1]))
            u = tf.unstack(u_trunk, axis=1)
        du_x = []
        #du_y = []
        for us in u:
        #u = tf.expand_dims(u, axis=-1)
            du = tf.unstack(g1.gradient(us, x), axis=1)
            du_x.append(du[0])
            #du_y.append(du[1])
        #du_x, du_y = tf.stack(du_x), tf.stack(du_y)
        du_x = tf.stack(du_x)
        #print(u_branch.shape, u_trunk.shape, du_x.shape)
        u_out = tf.tensordot(u_branch, u_trunk, axes=([1], [1]))
        du_x  = tf.tensordot(u_branch, du_x, axes=([1], [0]))
        #du_y  = tf.tensordot(u_branch, du_y, axes=([1], [0]))

        #return tf.tanh(u_out), tf.math.multiply(du_x, 1 - tf.math.multiply(tf.tanh(u_out), tf.tanh(u_out) )) 
        return u_out, du_x

    else:
        u_trunk = self.trunk(x)
        u_out = tf.tensordot(u_branch, u_trunk, axes=([1], [1]))
        
        return u_out
        #return tf.tanh(u_out)

  def get_config(self):
        return {"trunk": self.trunk, "branch": self.branch}

  @classmethod
  def from_config(cls, config):
      return cls(**config)

In [ ]:
class Data():
    '''du/dt=-ku(x,t), -1<=x<=1
        input u(x, t0)
       output u(x, t1)
    '''
    def __init__(self, x, sensor_in, sensor_out, length_scale, train_num, test_num):
        self.x = x
        self.sensor_in = sensor_in
        self.sensor_out = sensor_out
        self.length_scale = length_scale
        self.train_num = train_num
        self.test_num = test_num
        self.__init_data()
        
    def __init_data(self):
        
        features = 100
        u_train = gaussian_process(self.x, features, self.train_num, self.length_scale)
        u_test = gaussian_process(self.x, features, self.test_num,  self.length_scale)

        x0, x1 = self.x

        X = np.linspace(x0, x1, num=features, dtype=np.float32)

        u_train = 16*u_train*(1. - X)**2*X**2
        u_test = 16*u_test*(1. - X)**2*X**2

        u_train = self.zero_mean(u_train)
        u_test = self.zero_mean(u_test)

        u_train = normalize(u_train)
        u_test = normalize(u_test)

        self.u_train = tf.constant(u_train, dtype=tf.float32)
        self.u_test = tf.constant(u_test, dtype=tf.float32)
        self.X = tf.constant(np.reshape(X, [-1, 1]), dtype=tf.float32)


    def zero_mean(self, y):
            
            y = np.divide(y, np.reshape(np.max(np.abs(y), 1), (-1,1)) )
            
            return np.subtract(y, np.reshape(np.mean(y, 1), (-1,1)))

In [ ]:
# training data
Nx = 100
L = 1.

xd = [0., L]

num_sensor_in = 100
num_sensor_out = 20
length_scale_list = [0.2] #0. 2
num_train = 200
num_test = 1000

data_sensor = Data(xd, num_sensor_in, num_sensor_out, length_scale_list, num_train, num_test)

In [ ]:
tao = 5E-4  # time step
eps = 0.025 # Pysical constant

dx = L/(Nx-1) # spatial discretization for finite difference 
dt = 1e-6     # time step
N = int(tao/dt)

# Homogenous free energy density
Fe = lambda u: (u**2 - 1)**2/4

In [ ]:
# matrix to solve possion equation 
A = diags([1, -2, 1], [-1, 0, 1], (Nx, Nx)).toarray()
A[0,0], A[-1,-1] = -1, -1
invA = tf.constant(np.linalg.pinv(A), dtype=tf.float32)

In [ ]:
u_train = sense(data_sensor.u_train, num_sensor_in)
#u_out_train = solve_Cahn_Hilliard(u_train, eps, dt, dx, N)

u_test = sense(data_sensor.u_test, num_sensor_in)
u_out_test = solve_Cahn_Hilliard(u_test, eps, dt, dx, N)

In [ ]:
# Solve the time history of u at multiple time steps as the training data
# Every 2 time steps 
u_train_temp = u_train
u_out_temp = []
#u_out_temp.append(u_out_train)

for i in range(21):
    u_train_temp = solve_Cahn_Hilliard(u_train_temp, eps, dt, dx, N)
    if (i+1)%2 == 0:
        u_train = tf.concat([u_train, u_train_temp], axis=0)
    if (i+1)%2 == 1:
        u_out_temp.append(u_train_temp)

u_out_train = tf.concat(u_out_temp, axis=0)

In [ ]:
#save_object([u_train, u_out_train, u_test, u_out_test], 'Data/CH-1D_10000-500.pkl')
#u_train, u_out_train, u_test, u_out_test = load_object('Data/CH-1D_10000-500.pkl')

In [ ]:
# u initially sampled at 100 evenly spaced points
u_train_p = u_train
# down sample very 5 columns
u_train = u_train[:, ::5]
u_test = u_test[:, ::5]

In [ ]:
n_layer_trunk = 3
n_nodes_trunk =  100
n_layer_branch = 2
n_nodes_branch = 100
num_sensor_out = 100
Net_trunk = FNN(num_sensor_out, n_layer_trunk, n_nodes_trunk, 'relu')
Net_branch = FNN(num_sensor_out, n_layer_branch, n_nodes_branch, 'relu')
u_onet = ONet(Net_trunk, Net_branch)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((u_train, u_train_p))
train_dataset = train_dataset.shuffle(buffer_size=num_train).batch(256)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
# function to calulate the r2 score and mse
# samples are 3d matrix, with the first dimension being the sample number
def accuracy(u_pred, u_true):
    
    
    r2 = r2_score(u_true, u_pred)
    mse = mean_squared_error(u_true, u_pred)
    # L2 relative error
    rel = np.sqrt(np.sum((u_true - u_pred)**2)) / np.sqrt(np.sum(u_true**2))
    
    return r2, rel

In [ ]:
def physics_informed_train_step(onet, up, Xf, up0):
    
    with tf.GradientTape() as g:

        uq, du_x = onet(up, Xf, True)
        
        du = uq - up0
        df = tf.tensordot(- du * dx**2, invA, [[1], [1]])
        
        L_energy   = tf.reduce_mean(0.5*du_x**2 + 1./eps**2*Fe(uq)) * eps**2
        L_distance = tf.reduce_mean( df * du ) /tao/2.
        
        L_mean_u = tf.reduce_mean(uq)**2
        
        loss = L_energy + L_distance + 1e3 * L_mean_u
        

    grads = g.gradient(loss, onet.trainable_variables)

    optimizer.apply_gradients(zip(grads, onet.trainable_variables))  # + [lamda]
        
    return L_energy, L_distance, L_mean_u, loss

In [ ]:
%%time
nepochs = 3000
optimizer = tf.optimizers.Adam(learning_rate=0.001)

X = np.linspace(0, 1, Nx, dtype=np.float32)
X = tf.constant(X.reshape(-1,1), dtype=tf.float32)
 
for epoch in range(nepochs):

    for up, up0 in train_dataset:
 
        L_energy, L_distance, L_mean_u, loss = physics_informed_train_step(u_onet, up, X, up0)

    if (epoch+1) % 100 == 0:
        print(f'Epoch :{epoch+1}, Loss:{loss:.4e}, L_energy:{L_energy:.4e}, L_distance:{L_distance:.4e}, L_mean_u:{L_mean_u:.4e}')
        #print(f'Accuracy: Train - {r2_train:.4f},  Test - {r2_test:.4f}\n')
        u_test_pred = u_onet(u_test, X)
        r2_test, mse_test = accuracy(u_test_pred, u_out_test)
        
        print(f'Test r2 score:{r2_test:.4f}, Test mse:{mse_test:.4e}')

In [ ]:
train_dataset_sup = tf.data.Dataset.from_tensor_slices((u_train, u_out_train))
train_dataset_sup = train_dataset_sup.shuffle(buffer_size=num_train).batch(256)

In [ ]:
def suppervised_train_step(onet, up, Xf, uq):
    
    with tf.GradientTape() as g:

        uq_pred= onet(up, Xf)
        
        loss = tf.reduce_mean((uq_pred - uq)**2)
        

    grads = g.gradient(loss, onet.trainable_variables)

    optimizer.apply_gradients(zip(grads, onet.trainable_variables))  # + [lamda]
        
    return loss

In [ ]:
%%time
nepochs = 1000
optimizer = tf.optimizers.Adam(learning_rate=0.0002)

X = np.linspace(0, 1, Nx, dtype=np.float32)
X = tf.constant(X.reshape(-1,1), dtype=tf.float32)

for epoch in range(nepochs):

    for up, uq in train_dataset:

        loss = suppervised_train_step(u_onet, up, X, uq)

    if (epoch+1) % 200 == 0:

        u_test_pred = u_onet(u_train, X)
        r2_test, mse_test = accuracy(u_test_pred, u_out_train)

        print(f'Epoch :{epoch+1}, Loss:{loss:.4e}')
        print(f'Test r2 score:{r2_test:.4f}, Test mse:{mse_test:.4e}')
        

In [ ]:
# save weights only
# u_onet.save_weights('Saved_Models/CH-1D-NN-Map-weights')
# load weights only
# u_onet.load_weights('Saved_Models/CH-1D-NN-Map-weights')
# u_onet = tf.keras.models.load_model('Saved_Models/CH-1D-PI-Map', custom_objects={'ONet': ONet, 'FNN': FNN})

In [ ]:
up = np.array([np.cos(4*np.pi*x) for x in np.linspace(0., 1., 21)])
up = tf.constant(up.reshape(1, -1)[0:,0:-1], dtype=tf.float32)

uq = []
uq.append(up)
for i in range(200):
    up = u_onet(up, X)
    uq.append(up)
    up = up[:,::5]

In [ ]:
u0 = np.array([np.cos(4*np.pi*x) for x in np.linspace(0., 1., 101)])
u0 = tf.constant(u0.reshape(1, -1)[0:,0:-1], dtype=tf.float32)

u_exact = []
u_exact.append(u0)

for i in range(200):
    u0 = solve_Cahn_Hilliard(u0, eps, dt, dx, N)
    u_exact.append(u0)

In [ ]:

xlim = [0., 1.]
xticks = [0, 0.25, 0.5, 0.75, 1.]
ylim = [-1.05, 1.05]
yticks= [-1., -0.5, 0, 0.5, 1.]
xlabel = r'$x$'
ylabel = r'$c$'

fig, ax = plot_2D(xlim=xlim, ylim=ylim, xticks=xticks, yticks=yticks, xlabel=xlabel, ylabel=ylabel)
# ax.plot(X, u_exact[0][0], 'k', linewidth = 3, alpha=0.2, label='t = 0')

for i in [1]:
    u1 = uq[i][0]
    u1_exact = u_exact[i][0]
    
    #ax.plot(X, u1, 'ro', markevery=1, markersize=6, markeredgewidth=1, markerfacecolor=[1, 0, 0, 0.5])
    ax.plot(X, u1, 'r--', linewidth = 1.5)
    #ax.plot(x, u1_exact, 'r--', linewidth = 3, alpha= 0.99, label = 't = ' + str(i) + r"$\mathit{\tau}$ - Prediction")
    ax.plot(X, u1_exact, 'k', linewidth = 1.5) #,  label = 't = ' + str(i) + r"$\mathit{\tau}$")

u1 = uq[3][0]
u1_exact = u_exact[3][0]
    
#ax.plot(X, u1, 'ro', markevery=1, markersize=6, markeredgewidth=1, markerfacecolor=[1, 0, 0, 0.5])
ax.plot(X, u1, 'r--', linewidth = 1.5)
#ax.plot(x, u1_exact, 'r--', linewidth = 3, alpha= 0.99, label = 't = ' + str(i) + r"$\mathit{\tau}$ - Prediction")
ax.plot(X, u1_exact, 'k', linewidth = 1.5) #,  label = 't = ' + str(i) + r"$\mathit{\tau}$")

#ax.legend(fontsize='large', bbox_to_anchor=(0.88, 0.95))

#plt.savefig('CH_1D_Map-1.png', dpi=300, transparent=True)
#plt.show()